## 边缘检测的步骤
> * 滤波: 边缘检测的算法主要是基于图像强度的一阶和二阶导数，但导数通常对噪声很敏感，因此必须采用滤波器来改善与噪声有关的边缘检测器的性能。常见的滤波方法主要有高斯滤波，即采用离散化的高斯函数产生一组归一化的高斯核，然后基于高斯核函数对图像灰度矩阵的每一点进行加权求和

> * 增强: 增强边缘的基础是确定图像各点邻域强度的变化值。增强算法可以将图像灰度点邻域强度值有显著变化的点凸显出来。在具体编程实现时，可通过计算梯度幅值来确定

> * 检测: 经过增强的图像，往往邻域中有很多点的梯度值比较大，而在特定的应用中，这些点并不是我们要找的边缘点，所以应该采用某种方法来对这些点进行取舍。实际工程中，常用的方法是通过阈值化方法来检测

## [图像阈值](http://ex2tron.wang/2017/12/07/Python-OpenCV%E6%95%99%E7%A8%8B6%EF%BC%9A%E9%98%88%E5%80%BC%E5%88%86%E5%89%B2/)

### 简单阈值
> 整幅图像采用同一个数作为阈值（全局阈值）。像素值高于阈值时，给这个像素赋予一个新值（可能是白色），否则我们给它赋予另外一种颜色（也许是黑色）。这个函数就是 cv2.threshhold()。

>>函数原型:  
```
threshold(src, thresh, maxval, type[, dst]) -> retval, dst
```
>* 第一个参数就是原图像，原图像应该是灰度图  
>* 第二个参数就是用来对像素值进行分类的阈值  
>* 第三个参数就是当像素值高于（有时是小于）阈值时应该被赋予的新的像素值   
>* OpenCV提供了多种不同的阈值方法，这是有第四个参数来决定的。这些方法包括： 


>>阈值类型一般分为五种： 
```
cv2.THRESH_BINARY —— 大于阈值的部分像素值变为最大值，其他变为0   
cv2.THRESH_BINARY_INV —— 大于阈值的部分变为0，其他部分变为最大值   
cv2.THRESH_TRUNC —— 大于阈值的部分变为阈值，其余部分不变   
cv2.THRESH_TOZERO —— 大于阈值的部分不变，其余部分变为0   
cv2.THRESH_TOZERO_INV —— 大于阈值的部分变为0，其余部分不变  
```

### 自适应阈值
> 当同一幅图像上的不同部分的具有不同亮度时，这种情况下需要采用自适应阈值。此时的阈值是根据图像上的每一个小区域计算与其对应的阈值。因此在同一幅图像上的不同区域采用的是不同的阈值，从而能在亮度不同的情况下得到更好的结果


>>函数原型:
```
adaptiveThreshold(src, maxValue, adaptiveMethod, thresholdType, blockSize, C[, dst]) -> dst
```
>* 参数1：要处理的原图  
>* 参数2：最大阈值，一般为255  
>* 参数3：小区域阈值的计算方式  
     ADAPTIVE_THRESH_MEAN_C：小区域内取均值    
     ADAPTIVE_THRESH_GAUSSIAN_C：小区域内加权求和，权重是个高斯核
>* 参数4：阈值方式（跟前面讲的那5种相同）
>* 参数5：邻域的面积，如11就是11x11的小块
>* 参数6：一个常数，最终阈值等于小区域计算出的阈值再减去此值

### [Otsu's 二值化](http://ex2tron.wang/2017/12/08/Python-OpenCV%E6%95%99%E7%A8%8B%E7%95%AA%E5%A4%96%E7%AF%874%EF%BC%9AOtsu%E9%98%88%E5%80%BC%E6%B3%95/)
> 如果是一幅双峰图像（双峰图像是指图像直方图中存在两个峰）应该在两个峰之间的峰谷选一个值作为阈值。这就是 Otsu 二值化要做的。简单来说就是对一幅双峰图像自动根据其直方图计算出一个阈值。（对于非双峰图像，这种方法得到的结果可能会不理想）

> 用到到的函数还是 cv2.threshold()，但是需要多传入一个参数（flag）： cv2.THRESH_OTSU。这时要把阈值设为 0。然后算法会找到最优阈值，这个最优阈值就是返回值 retVal。如果不使用 Otsu 二值化，返回的retVal 值与设定的阈值相等

## [滤波和模糊](http://ex2tron.wang/2017/12/15/Python-OpenCV%E6%95%99%E7%A8%8B10%EF%BC%9A%E5%B9%B3%E6%BB%91%E5%9B%BE%E5%83%8F/)

> 它们都属于卷积，不同滤波方法之间只是卷积核不同（对线性滤波而言）  
低通滤波器（lowpass）是模糊，减弱或阻隔高频信号，保留低频信号    
高通滤波器（highpass）是锐化，减弱或阻隔低频信号，保留高频信号  

>> 低通滤波器就是允许低频信号通过，在图像中边缘和噪点都相当于高频部分，所以低通滤波器用于去除噪点、平滑和模糊图像。
高通滤波器则反之，用来增强图像边缘，进行锐化处理

>> 常见噪声有椒盐噪声和高斯噪声，椒盐噪声可以理解为斑点，随机出现在图像中的黑点或白点；高斯噪声可以理解为拍摄图片时由于光照等原因造成的噪声；这样解释并不准确，只要能简单分辨即可

> 使用低通滤波器可以达到图像模糊的目的。这对与去除噪音很有帮助。其实就是去除图像中的高频成分（比如：噪音，边界）。所以边界也会被模糊一点。（当然，也有一些模糊技术不会模糊掉边界）。 OpenCV 提供了四种模糊技术

### 均值滤波
> 均值滤波是一种最简单的滤波处理，它取的是卷积核区域内元素的均值，用cv2.blur()实现：

>> 函数原型:
```python
blur(src, ksize[, dst[, anchor[, borderType]]]) -> dst
```

>> 示例:
```python
img = cv2.imread('img/lena.jpg')
img = cv2.blur(img, (3, 3))   # 使用 3×3的卷积核
```

### 方框滤波
> 方框滤波跟均值滤波很像，用cv2.boxFilter()函数实现，事实上，当可选参数normalize为True的时候，方框滤波就是均值滤波；normalize为False的时候，相当于求区域内的像素和

>> 函数原型:
```python
boxFilter(src, ddepth, ksize[, dst[, anchor[, normalize[, borderType]]]]) -> dst
```

>> 示例:
```python
* 前面的均值滤波也可以用方框滤波实现：normalize=True
blur = cv2.boxFilter(img, -1, (3, 3), normalize=True)
```

### 高斯滤波
> 把卷积核换成高斯核（简单来说，方框不变，将原来每个方框的值是相等的，现在里面的值是符合高斯分布的，方框中心的值最大，其余方框根据距离中心元素的距离递减，构成一个高斯小山包。原来的求平均数现在变成求加权平均数，全就是方框里的值）。实现的函数是 cv2.GaussianBlur()。需要指定高斯核的宽和高（必须是奇数）。以及高斯函数沿 X， Y 方向的标准差。如果只指定了 X 方向的的标准差， Y 方向也会取相同值。如果两个标准差都是 0，那么函数会根据核函数的大小自己计算。高斯滤波可以有效的从图像中去除高斯噪音

>> 函数原型:
```python
GaussianBlur(src, ksize, sigmaX[, dst[, sigmaY[, borderType]]]) -> dst
```
>> 示例:
```python
img = cv2.GaussianBlur(img, (5, 5), 1) # 使用 5×5的高斯核
```

> 注：高斯滤波相比均值滤波效率要慢，但可以有效消除高斯噪声，能保留更多的图像细节，所以经常被称为最有用的滤波器。

### 中值滤波
> 中值又叫中位数，是所有值排序后取中间的值。中值滤波就是用区域内的中值来代替本像素值，所以那种孤立的斑点，如0或255很容易消除掉，适用于去除椒盐噪声和斑点噪声。中值是一种非线性操作，效率相比前面几种线性滤波要慢。

>> 函数原型:
```python
medianBlur(src, ksize[, dst]) -> dst
```
>> 示例:
```python
img = cv2.medianBlur(img, 5)   
```

### 双边滤波
> 已知道高斯滤波器是求中心点邻近区域像素的高斯加权平均值。这种高斯滤波器只考虑像素之间的空间关系，而不会考虑像素值之间的关系（像素的相似度）。所以这种方法不会考虑一个像素是否位于边界，因此边界也会别模糊掉。
函数 cv2.bilateralFilter() 能在保持边界清晰的情况下有效的去除噪音，但是这种操作与其他滤波器相比会比较慢；双边滤波在同时使用`*空间高斯权重*`和`*灰度值相似性高斯权重*`；空间高斯函数确保只有邻近区域的像素对中心点有影响，灰度值相似性高斯函数确保只有与中心像素灰度值相近的才会被用来做模糊运算。所以这种方法会确保边界不会被模糊掉，因为边界处的灰度值变化比较大

>> 函数原型:
```python
bilateralFilter(src, d, sigmaColor, sigmaSpace[, dst[, borderType]]) -> dst
```
>> 示例:
```python
img = cv2.bilateralFilter(img, 9, 75, 75)   
```

### 小结
* 在不知道用什么滤波器好的时候，优先高斯滤波 cv2.GaussianBlur()，然后均值滤波 cv2.blur()
* 斑点和椒盐噪声优先使用中值滤波 cv2.medianBlur()
* 要去除噪点的同时尽可能保留更多的边缘信息，使用双边滤波 cv2.bilateralFilter()
* 线性滤波方式：均值滤波、方框滤波、高斯滤波（速度相对快）
* 非线性滤波方式：中值滤波、双边滤波（速度相对慢）